In [25]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
import lightgbm as lgb

from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression



Requirements.txt for now:

pandas

xgboost 

scikit-learn

numpy

streamlit

# Preprocessing

Let's think about it in a way that it will be a function at some point 

The target is a numerical value whic is the purchase

In [26]:
train_path = r'data\train.csv'
train_df = pd.read_csv(train_path)
train_df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [27]:
print (train_df.columns)

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')


In [28]:
#first thing is copy whole data set and then drop the User_ID and Product_ID
train_data_copy = train_df.copy()
#this will be used in the end
train_data = train_df.copy()

ID_columns = ["User_ID", "Product_ID"]
train_data_copy = train_data_copy.drop(ID_columns, axis=1)
train_data_copy.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,M,55+,16,C,4+,0,8,NaN,NaN,7969


## Null Values Count 

This will help getting an idea which will be obligatory and which will be optional when doing a web app for forms 

In [29]:
train_data_copy.isna().sum()

Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

### Analysis 

For products 0-20

Category 1 which was associated with clothes is obligatory between 1 and 18

Category 2 assciated with electronics will be optional and it is between 2-18

Category 3 associated with home goods and it is between 3-18 and it is optional

For Occupation is between 0-20

## Categorical Values & Numerical Values 

Categorical: 

Age (0-17, 18-25, 26-35, 36-45, 46-50, 51-55, 55+)

Gender (M, F)

City_Category (A, B, C)

Stay In Current City (0,1,2,3,4+)

In [30]:
numerical_features = ["Occupation", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]
categorical_features = ["Age", "Gender", "City_Category", "Stay_In_Current_City_Years"]

X = train_data_copy[numerical_features + categorical_features]
y = train_data_copy["Purchase"]

print (y.head())

X.head()

0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64


,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Age,Gender,City_Category,Stay_In_Current_City_Years
0,10,0,3,NaN,NaN,0-17,F,A,2
1,10,0,1,6.0,14.0,0-17,F,A,2
2,10,0,12,NaN,NaN,0-17,F,A,2
3,10,0,12,14.0,NaN,0-17,F,A,2
4,16,0,8,NaN,NaN,55+,M,C,4+


# Steps

X will have the numerical values checked for null values scaled and 

then after train, test, split one hot encoding on the categorical features will be done 

The goal is to use XGboost which handles Nan Values. 

This is due since it will affect the y value if we took the mean 

In [31]:
mean_product_3 = X["Product_Category_3"].mean()
mean_product_2 = X["Product_Category_2"].mean()
print (f"Mean for the Category product 2: {mean_product_2}\nand for category 3: {mean_product_3}")

print()
median_product_3 = X["Product_Category_3"].median()
median_product_2 = X["Product_Category_2"].median()
print (f"Median for the Category product 2: {median_product_2}\nand for category 3: {median_product_3}")

print()
mode_product_3 = X["Product_Category_3"].mode()
mode_product_2 = X["Product_Category_2"].mode()
print (f"Mode for the Category product 2: {mode_product_2}\nand for category 3: {mode_product_3}")


Mean for the Category product 2: 9.842329251122386
and for category 3: 12.668243206790512

Median for the Category product 2: 9.0
and for category 3: 14.0

Mode for the Category product 2: 0    8.0
Name: Product_Category_2, dtype: float64
and for category 3: 0    16.0
Name: Product_Category_3, dtype: float64


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ordinal_encoder = OrdinalEncoder()

X_train_cat_encoded = ordinal_encoder.fit_transform(X_train[categorical_features])
X_test_cat_encoded = ordinal_encoder.transform(X_test[categorical_features]) 

ordinal_encoder.fit(X_train[categorical_features])
X_train_cat_encoded = ordinal_encoder.transform(X_train[categorical_features])
X_test_cat_encoded = ordinal_encoder.transform(X_test[categorical_features])

X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded, columns=categorical_features, index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=categorical_features, index=X_test.index)

X_train_final = pd.concat([X_train[numerical_features], X_train_cat_encoded_df], axis=1)
X_test_final = pd.concat([X_test[numerical_features], X_test_cat_encoded_df], axis=1)

# Model Training, fitting and Accuracy

In [33]:
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train_final,y_train)
y_pred_xgb = xgb_model.predict(X_test_final)

y_pred_xgb_abs = np.abs(y_pred_xgb)

r_squared_xgb= r2_score(y_test, y_pred_xgb_abs)
print(f"R squared: {r_squared_xgb}")

R squared: 0.6671998884222712


In [34]:
lgb_model = lgb.LGBMRegressor(objective='regression',random_state=42)
lgb_model.fit(X_train_final, y_train)
y_pred_lgb = lgb_model.predict(X_test_final)
y_pred_lgb_abs = np.abs (y_pred_lgb)

r_squared_lgb=r2_score(y_test, y_pred_lgb_abs)

print(f"r2 squared for lgb {r_squared_lgb}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 97
[LightGBM] [Info] Number of data points in the train set: 440054, number of used features: 9
[LightGBM] [Info] Start training from score 9266.733955
r2 squared for lgb 0.6597296575153713


# Sanity Checking & Preprocessing Function/s

In [35]:
#repititon just for the sake of the 
numerical_features_prep = ["Occupation", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]
categorical_features_prep = ["Age", "Gender", "City_Category", "Stay_In_Current_City_Years"]
ID_columns_prep = ["User_ID", "Product_ID"]
"""
Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')
"""
obligatory_data = ["User_ID", "Product_ID", "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status","Product_Category_1"]
out_put = ["Purchase"]
optional_data = ["Product_Category_2", "Product_Category_3"]

def checking_null_sanity(input_data: pd.DataFrame, is_inference: bool):
    # Assuming obligatory_data and out_put are defined elsewhere in your code
    check_columns = obligatory_data + (out_put if not is_inference else [])
    null_rows_df = input_data[check_columns].isnull().any(axis=1)
    df_with_nulls = input_data[null_rows_df]
    condition = (
        input_data['Gender'].isin(['M', 'F']) &
        input_data['Product_Category_1'].between(1, 18) &
        (input_data['Product_Category_2'].isnull() | input_data['Product_Category_2'].between(2, 18)) &
        (input_data['Product_Category_3'].isnull() | input_data['Product_Category_3'].between(3, 18)) &
        input_data['City_Category'].isin(['A', 'B', 'C']) &
        input_data['Marital_Status'].isin([0, 1]) &
        input_data['Occupation'].between(0, 20)
    )
    if not is_inference:
        condition &= (input_data[out_put[0]] > 0)
    df_with_value_issues = input_data[~condition]
    if is_inference is False:
        negative_values=input_data[out_put[0]]<=0
        df_negative_values=input_data[negative_values]
        df_issues=pd.concat([df_with_nulls, df_with_value_issues,df_negative_values]).drop_duplicates()
    else:
        df_issues = pd.concat([df_with_nulls, df_with_value_issues]).drop_duplicates()
    input_data_cleaned = input_data.drop(df_issues.index)
    """
    df_issues will then be added to the database as quality issues 
    """
    return df_issues, input_data_cleaned



def preprocessing(input_data: pd.DataFrame, is_inference: bool):
    ordinal_encoder = OrdinalEncoder()
    input_data = input_data.drop(ID_columns_prep, axis=1)
    
    if not is_inference:
        X = input_data.drop(out_put, axis=1)
        y = input_data[out_put[0]]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        ordinal_encoder.fit(X_train[categorical_features_prep])
        X_train_cat_encoded = ordinal_encoder.transform(X_train[categorical_features_prep])
        X_test_cat_encoded = ordinal_encoder.transform(X_test[categorical_features_prep])
        X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded, columns=categorical_features_prep, index=X_train.index)
        X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=categorical_features_prep, index=X_test.index)
        X_train_final = pd.concat([X_train[numerical_features_prep], X_train_cat_encoded_df], axis=1)
        X_test_final = pd.concat([X_test[numerical_features_prep], X_test_cat_encoded_df], axis=1)
        return X_train_final, X_test_final, y_train, y_test  
    else:
        X_test_cat_encoded = ordinal_encoder.transform(input_data[categorical_features_prep])
        X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=categorical_features_prep, index=input_data.index)
        X_test_final = pd.concat([input_data[numerical_features_prep], X_test_cat_encoded_df], axis=1) 
        return X_test_final


In [36]:
def xgb_model_func(X_train,X_test,y_train,y_test):
    xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
    xgb_model.fit(X_train,y_train)
    
    y_pred_xgb = xgb_model.predict(X_test)
    y_pred_xgb_abs = np.abs(y_pred_xgb)
    r_squared_xgb= r2_score(y_test, y_pred_xgb_abs)
    print(f"R squared: {r_squared_xgb}")
    return dict({"r-squared": r_squared_xgb})

In [55]:
def elastic_net_func(X_train,X_test,y_train,y_test,alpha=0.1,l1_ratio=0.5):
    elastic_net=ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    elastic_net.fit(X_train,y_train)
    y_pred_en=elastic_net.predict(X_test)
    r_squared=r2_score(y_test,y_pred_en)
    print(f"R squared: {r_squared}")
    return dict({"r_squared": r_squared})


# Retesting with null values as median

The goal is that we need to monitor the median of such values

Null values are only in the "Product_Category_2", "Product_Category_3"

In [38]:
def replacing_null_median(data:pd.DataFrame):
    if ('Product_Category_2' in data.columns) and ('Product_Category_3' in data.columns):
        median_product_3 = data["Product_Category_3"].median()
        median_product_2 = data["Product_Category_2"].median()
        data["Product_Category_2"].fillna(median_product_2, inplace=True)
        data["Product_Category_3"].fillna(median_product_3, inplace=True)
        return data
    else: 
        print ("one or both columns are missing")
        return None 

In [39]:
X_without_null = replacing_null_median(X)
X_without_null.isnull().sum()

C:\Users\SADEK COMPUTER\AppData\Local\Temp\ipykernel_24280\3709695468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Product_Category_2"].fillna(median_product_2, inplace=True)
C:\Users\SADEK COMPUTER\AppData\Local\Temp\ipykernel_24280\3709695468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Product_Category_3"].fillna(median_product_3, inplace=True)


Occupation                    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Age                           0
Gender                        0
City_Category                 0
Stay_In_Current_City_Years    0
dtype: int64

# Testing the previously done Functions on Train_data

In [40]:
train_data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


### Testing Sanity Function

In [41]:
train_data_issues,train_data_check = checking_null_sanity(train_data, False)
train_data_issues.head(13)

#train_data_issues.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
545915,1000001,P00375436,F,0-17,10,A,2,0,20,NaN,NaN,612
545916,1000002,P00372445,M,55+,16,C,4+,0,20,NaN,NaN,119
545917,1000004,P00375436,M,46-50,7,B,2,1,20,NaN,NaN,481
545918,1000006,P00375436,F,51-55,9,A,1,0,20,NaN,NaN,480
545919,1000007,P00372445,M,36-45,1,B,1,1,20,NaN,NaN,241
545920,1000008,P00370293,M,26-35,12,C,4+,1,19,NaN,NaN,48
545921,1000009,P00375436,M,26-35,17,C,0,0,20,NaN,NaN,139
545922,1000010,P00370293,F,36-45,1,B,4+,1,19,NaN,NaN,24
545923,1000011,P00371644,F,26-35,1,C,1,0,20,NaN,NaN,121
545924,1000015,P00375436,M,26-35,7,A,1,0,20,NaN,NaN,604


In [42]:
train_data_check.head(13)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [43]:
train_data_check.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            169485
Product_Category_3            379094
Purchase                           0
dtype: int64

### Testing Median Changing function

In [44]:
train_data_check_no_null = replacing_null_median(train_data_check)
train_data_check_no_null.isnull().sum()

User_ID                       0
Product_ID                    0
Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Product_Category_3            0
Purchase                      0
dtype: int64

### Testing the preprocessing function and model

In [45]:
X_train,X_test,y_train,y_test=preprocessing(train_data_check_no_null,False)

In [50]:
xgb_mode = xgb_model_func(X_train,X_test,y_train,y_test)
print(xgb_mode)

R squared: 0.6619969469919352
{'r-squared': 0.6619969469919352}


In [56]:
el_net=elastic_net_func(X_train,X_test,y_train,y_test,alpha=1.0, l1_ratio=0.2)
print(el_net)

R squared: 0.10466244224293886
{'r_squared': 0.10466244224293886}


## Making Inference